<h3>Using pandas for data processing, numpy, sklearn for split train and test and created package called learn</h3>

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from learn.classifier import KNN
from learn.decomposition import PCA
from learn.metrics import accuracy

<br/>
Original data contains 1000 audio tracks with 10 genres (100 tracks each genre) and 50 extracted feature (mfcc, chroma stft and rmse). There are 12 variable for mfcc and stft and 1 variable for rmse (total 25 feature), for each these features we calculated mean and std producing total 50 features. Together with labels/genres/Target, the original data contains 51 columns and 1000 rows.

In [11]:
df = pd.read_csv('./data/genres.csv')
print(df.shape)

(1000, 51)


In [12]:
df['Target'].unique()

array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
       'metal', 'pop', 'reggae', 'rock'], dtype=object)

In [13]:
df.columns

Index(['Target', 'chroma_stft_mean_1', 'chroma_stft_mean_10',
       'chroma_stft_mean_11', 'chroma_stft_mean_12', 'chroma_stft_mean_2',
       'chroma_stft_mean_3', 'chroma_stft_mean_4', 'chroma_stft_mean_5',
       'chroma_stft_mean_6', 'chroma_stft_mean_7', 'chroma_stft_mean_8',
       'chroma_stft_mean_9', 'chroma_stft_std_1', 'chroma_stft_std_10',
       'chroma_stft_std_11', 'chroma_stft_std_12', 'chroma_stft_std_2',
       'chroma_stft_std_3', 'chroma_stft_std_4', 'chroma_stft_std_5',
       'chroma_stft_std_6', 'chroma_stft_std_7', 'chroma_stft_std_8',
       'chroma_stft_std_9', 'mfcc_mean_1', 'mfcc_mean_10', 'mfcc_mean_11',
       'mfcc_mean_12', 'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4',
       'mfcc_mean_5', 'mfcc_mean_6', 'mfcc_mean_7', 'mfcc_mean_8',
       'mfcc_mean_9', 'mfcc_std_1', 'mfcc_std_10', 'mfcc_std_11',
       'mfcc_std_12', 'mfcc_std_2', 'mfcc_std_3', 'mfcc_std_4', 'mfcc_std_5',
       'mfcc_std_6', 'mfcc_std_7', 'mfcc_std_8', 'mfcc_std_9', 'rmse_mean_1',
 

<br/>
We want to get best combination of genres, because using all genres produce bad accuracy. For each combination of genres, we tested the accuracy. Accuracy calculated with KNN classifier, but before fit to model, we transform the data with PCA. Transformed data contains only 5 columns/features.

In [14]:
def test_accuracies(genres):
    df_selected = df[df['Target'].isin(genres)]
    X = df_selected.drop(columns="Target")
    y = df_selected["Target"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    pca = PCA(n_components=5)
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)
    
    clf = KNN(k=len(genres))
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = round(accuracy(y_test, y_pred) * 100, 3)
    print("Accuracy : {0}%,\tgenres : {1}".format(acc, ", ".join(genres)))

<br/>
Experiments for testing each combination of genres

In [15]:
all_genres = np.array([
    'blues', 
    'classical', 
    'country', 
    'disco', 
    'hiphop', 
    'jazz',
    'metal', 
    'pop', 
    'reggae', 
    'rock'
])

In [16]:
genres_combination = [
    list(all_genres[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]),
    list(all_genres[[0, 1, 2, 3, 4, 5, 6, 7, 8]]),
    list(all_genres[[0, 1, 2, 3, 4, 5, 6, 7]]),
    list(all_genres[[0, 1, 2, 3, 4, 5, 6]]),
    list(all_genres[[0, 1, 2, 3, 4, 5]]),
    list(all_genres[[0, 1, 2, 3, 5, 6]]),
    list(all_genres[[1, 2, 4, 5, 6]]),
    list(all_genres[[0, 1, 2, 5, 6, 7]]),
    list(all_genres[[3, 4, 5, 7, 8, 9]]),
    list(all_genres[[2, 3, 4, 5, 7, 8, 9]]),
]

for genre in genres_combination:
    test_accuracies(genre)

Accuracy : 43.667%,	genres : blues, classical, country, disco, hiphop, jazz, metal, pop, reggae, rock
Accuracy : 46.296%,	genres : blues, classical, country, disco, hiphop, jazz, metal, pop, reggae
Accuracy : 57.083%,	genres : blues, classical, country, disco, hiphop, jazz, metal, pop
Accuracy : 56.667%,	genres : blues, classical, country, disco, hiphop, jazz, metal
Accuracy : 53.889%,	genres : blues, classical, country, disco, hiphop, jazz
Accuracy : 60.556%,	genres : blues, classical, country, disco, jazz, metal
Accuracy : 76.0%,	genres : classical, country, hiphop, jazz, metal
Accuracy : 71.111%,	genres : blues, classical, country, jazz, metal, pop
Accuracy : 47.222%,	genres : disco, hiphop, jazz, pop, reggae, rock
Accuracy : 42.381%,	genres : country, disco, hiphop, jazz, pop, reggae, rock


<br/>
Best accuracy is when only using classical, country, hiphop, jazz, metal 

In [17]:
df_final = df[df['Target'].isin(['classical', 'country', 'hiphop', 'jazz', 'metal'])]
df_final.shape

(500, 51)

In [18]:
df_final.to_csv('./data/final_genres.csv', index=False)